<a href="https://colab.research.google.com/github/mugalan/working/blob/main/Projects_2026.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Overview


This notebook implements a hierarchical, multi-level directed graph modeling class called `HierarchicalGraph`. It supports rich metadata on both nodes and edges, multi-edge support, clustered + colored visualization, graph merging, selective subgraph viewing, and structured attribute extraction.

## Core Concepts

|           Level | Meaning                                                                                                                                              | Implemented as                                                    |
| --------------: | ---------------------------------------------------------------------------------------------------------------------------------------------------- | ----------------------------------------------------------------- |
| **Inner Graph** | Detailed node-to-node directed relations (supports parallel edges)                                                                                   | `networkx.MultiDiGraph()`                                         |
| **Outer Graph** | **Roots-only** abstraction (top-level groups). Each inner edge `u→v` is **lifted** to `root(u)→root(v)`; edges where `root(u)==root(v)` are omitted. | `graphviz` rendering from the inner graph (no extra parent nodes) |

### Hierarchy

* Hierarchical containment is defined by the **`parent`** attribute on nodes.
* A node with `parent=None` (or absent) is a **root** (top-level group/container).
* Inner visualization renders clusters (subgraphs) for parents **without drawing extra parent nodes**, and edges to/from parents are attached to their clusters (so you don’t get stray grey duplicates).

### Colors

* You can set `color` per node. If omitted, visualization falls back to white.
* You may still pass a palette on construction; apply it as you prefer in normalization (optional).

## Example Node Format

```python
nodes = [
    {'label': 'Group A', 'type': 'system', 'color': 'yellow', 'description': 'Top A'},
    {'label': 'A', 'parent': 'Group A', 'type': 'system', 'color': 'yellow'},
    {'label': 'B', 'parent': 'Group A', 'type': 'user',   'color': 'lightblue'},
]
```

> Required: `label`
> Optional: `parent`, `type`, `color`, `description`, and any other attributes (preserved).

## Example Edge Format

```python
edges = [
    {'start': 'A', 'end': 'B', 'type': 'control', 'weight': 0.4, 'color': 'blue'}
]
```

> Required: `start`, `end`
> Optional: `type`, `weight`, `color`, `description`, plus arbitrary extra attributes.
> Parallel edges (same `start`/`end` but different `type` or attrs) are supported.

## Usage Example

```python
hg = HierarchicalGraph(nodes, edges)

# Inner view: clustered, hierarchical, multi-edge
hg.visualize_inner_graph_with_clusters()

# Outer view: roots only; edges are lifted to root(u) -> root(v)
hg.visualize_outer_graph()
```

## Outer View — What You’ll See

* **Nodes:** only **root** nodes (those without a `parent`).
* **Edges:** every inner edge `u→v` is mapped to `root(u)→root(v)`.

  * If `root(u) == root(v)`, the edge is **suppressed** in the outer view (it’s an internal detail).
  * Multiple lifted edges between the same roots are allowed (shows parallel cross-group relations).
* **Colors:** taken from the root nodes’ `color` attribute (fallback: white).

## Merge Multiple Graphs

```python
hg1 = HierarchicalGraph(nodes1, edges1)
hg2 = HierarchicalGraph(nodes2, edges2)

hg1.merge(hg2)
```

* Node merging is by `label`.
* Edge merging distinguishes edges by the **triple key** `(start, end, type)`. Matching edges are updated; new ones are appended.

## Subgraph Visualization

```python
hg.visualize_subgraph(['A', 'C', 'F'])
```

Renders the induced subgraph for the provided labels (silently ignores unknown nodes), with colors inherited when available.

## Attribute Fetching

```python
hg.get_node_attributes()
hg.get_edge_attributes()
```

Returns consolidated attributes (implementation-specific helpers).

---

## 📤 CSV Export & Import

This class supports exporting and importing node and edge data using CSV files. This is useful for persistence, versioning, sharing, or editing data externally (e.g., in Excel).

### 🔄 Export Methods

```python
hg.export_nodes_to_csv("nodes.csv")
hg.export_edges_to_csv("edges.csv")
```

* **`export_nodes_to_csv(filename)`** — Saves all current node attributes.
* **`export_edges_to_csv(filename)`** — Saves all edge relationships and attributes.

### 🔄 Import Methods

```python
hg.import_nodes_from_csv("nodes.csv")
hg.import_edges_from_csv("edges.csv")
```

* **`import_nodes_from_csv(filename)`** — Loads node data, validates/normalizes, rebuilds graphs.
* **`import_edges_from_csv(filename)`** — Loads edge data, validates, rebuilds graphs.

> After importing, the internal graphs are **automatically rebuilt**.

### 📝 CSV Structure

#### Nodes CSV (`nodes.csv`)

Must include at minimum:

* `label` (required) — Unique node identifier
* `parent` (optional) — Parent node label (for hierarchy)
* Any other attributes are allowed (e.g., `color`, `type`, `description`, etc.)

#### Edges CSV (`edges.csv`)

Must include at minimum:

* `start` (required) — Source node label
* `end` (required) — Target node label
* Any other attributes are allowed (e.g., `type`, `weight`, `color`, `description`, etc.)


# Python Package Installation

In [ ]:
import json, csv
import pandas as pd
import networkx as nx
import graphviz
from IPython.display import display, Image

from pydantic import BaseModel, ValidationError, field_validator
from typing import Optional

In [ ]:
!pip install "git+https://github.com/mugalan/hierarchical-graphs.git"

In [ ]:
from hierarchical_graph import HierarchicalGraph

#Grpah Create/Edit

In [ ]:
nodes = [
    {'label': 'Group A', 'type': 'system', 'color':'yellow','description':'This Top A'},
    {'label': 'Group B', 'type': 'system', 'color':'lightgrey','description':'This Top B'},
    {'label': 'Group C', 'type': 'system', 'color':'lightgreen','description':'This Top C'},
    {'label': 'A', 'parent': 'Group A', 'type': 'system', 'color':'orange','description':'This node does this'},
    {'label': 'B', 'parent': 'Group A', 'type': 'user','color':'orange','description':'This node does this'},
    {'label': 'C', 'parent': 'Group B', 'type': 'system','color':'orange','description':'This node does B'},
    {'label': 'D', 'parent': 'Group B', 'type': 'user','color':'green','description':'This node does B'},
    {'label': 'E', 'parent': 'Group C', 'type': 'system','color':'pink','description':'This node does c'},
    {'label': 'F', 'parent': 'A', 'type': 'system','color':'pink','description':'This node does c'},
    {'label': 'G', 'parent': 'B', 'type': 'system','color':'pink','description':'This node does c'},
]

edges = [
    {'start': 'A', 'end': 'B', 'type': 'type1', 'weight': 0.4, 'color':'blue','description':'This edge is type-1'},
    {'start': 'A', 'end': 'C', 'type': 'type2', 'weight': 0.8,'description':'This edge is type-2'},
    {'start': 'B', 'end': 'C', 'type': 'type1', 'weight': 0.5,'description':'This edge is type-1'},
    {'start': 'C', 'end': 'E', 'type': 'type3', 'weight': 0.1,'description':'This edge is type-3'},
    {'start': 'C', 'end': 'E', 'type': 'type1', 'weight': 0.9}, # Multi-edge here
    {'start': 'D', 'end': 'E', 'type': 'type2', 'weight': 0.3},
    {'start': 'E', 'end': 'F', 'type': 'type4', 'weight': 0.3},
    {'start': 'F', 'end': 'G', 'type': 'type4', 'weight': 0.3},
]

In [ ]:
hg = HierarchicalGraph(nodes, edges)
hg.visualize_inner_graph_with_clusters()
hg.visualize_outer_graph()
hg.visualize_outer_graph_with_parent()

In [ ]:
hg.get_node_attributes(labels=["A","F"])

In [ ]:
hg.get_edge_attributes(edge_tuples=[('A','B'),('C','E')])

In [ ]:
hg.add_nodes({'label': 'F', 'group': 'Group A', 'type': 'user'})

In [ ]:
hg.add_edges(edge_data=[
    {'start': 'C', 'end': 'A', 'type': 'type5', 'weight': 0.4},
    {'start': 'F', 'end': 'E', 'type': 'type5', 'weight': 0.4}
])

In [ ]:
hg.visualize_inner_graph_with_clusters()

In [ ]:
hg.visualize_subgraph(["A","B","E"])

##Merge two graphs

In [ ]:
nodes2 = [
    {'label': 'Group A', 'type': 'system', 'color':'yellow','description':'This Top A'},
    {'label': 'Group B', 'type': 'system', 'color':'lightgrey','description':'This Top B'},
    {'label': 'Group C', 'type': 'system', 'color':'lightgreen','description':'This Top C'},
    {'label': 'Group D', 'type': 'system', 'color':'lightblue','description':'This Top C'},
    # existing node modifications
    {'label': 'A', 'parent': 'Group A', 'status':'deprecated'},   # adds new attrib to A
    {'label': 'C', 'parent': 'Group B', 'description':'updated desc for C'}, # override / update
    # new nodes
    {'label': 'F', 'parent': 'Group C', 'type':'ai-agent', 'color':'lightgreen','description':'New intelligence asset'},
    {'label': 'G', 'parent': 'Group D', 'type':'sensor', 'color':'lightcyan','description':'brand new group D node'},
    {'label': 'H', 'parent': 'Group D', 'type':'system', 'color':'lightcyan','description':'new second D member'}
]
edges2 = [
    # new edges between existing nodes
    {'start':'B','end':'D','type':'type7','weight':0.66,'color':'purple','description':'cross link BA'},
    {'start':'E','end':'A','type':'type2','weight':0.42,'description':'reverse feedback to A'},

    # update an existing edge (higher weight + new description)
    {'start':'A','end':'C','type':'type2','weight':0.92,'description':'updated weight for this relationship'},

    # edges using newly introduced nodes
    {'start':'A','end':'F','type':'typeX','weight':0.30},
    {'start':'F','end':'C','type':'typeX','weight':0.70},
    {'start':'G','end':'H','type':'internal','weight':0.22},
    {'start':'H','end':'C','type':'external','weight':0.44}
]

In [ ]:
hg2 = HierarchicalGraph(nodes2, edges2)
hg2.visualize_inner_graph_with_clusters()
hg2.visualize_outer_graph()

In [ ]:
hg.merge(hg2)

In [ ]:
hg.visualize_inner_graph_with_clusters()

## Export/Import

In [ ]:
hg.export_json('hg.json')

In [ ]:
hg_loaded = HierarchicalGraph.load_from_json("hg.json")
hg_loaded.visualize_inner_graph_with_clusters()

In [ ]:
hg.export_nodes_to_csv('nodes.csv')
hg.export_edges_to_csv('edges.csv')

In [ ]:
hg22=HierarchicalGraph([], [])

In [ ]:
hg22.import_nodes_from_csv('nodes.csv')
hg22.import_edges_from_csv('edges.csv')

In [ ]:
hg22.visualize_inner_graph_with_clusters()

In [ ]:
hg.edges_data

In [ ]:
hg22.edges_data

#Maintenance Management

## References

### Scientific Literature using Graph / Knowledge Graph Approaches in Industrial Maintenance

1. **Xia et al. (2023)**  
   *Maintenance planning recommendation of complex industrial equipment based on knowledge graph and graph neural network*  
   [Reliability Engineering & System Safety, Vol 232](https://doi.org/10.1016/j.ress.2022.109068)  
   DOI: 10.1016/j.ress.2022.109068

2. **Lou et al. (2023)**  
   *Knowledge Graph Construction Based on a Joint Model for Equipment Maintenance*  
   [Mathematics, 11(17): 3748](https://www.mdpi.com/2227-7390/11/17/3748)  
   DOI: 10.3390/math11173748

3. **Teern et al. (2022)**  
   *Knowledge graph construction and maintenance process: Design challenges for industrial maintenance support*  
   [CEUR Workshop Proceedings (PDF)](https://www.researchgate.net/publication/363926032_Knowledge_graph_construction_and_maintenance_process_Design_challenges_for_industrial_maintenance_support)

4. **Stewart et al. (2024)**  
   *MWO2KG and Echidna: Constructing and exploring an interactive maintenance knowledge graph*  
   [Journal of Maintenance & Innovation (DOI)](https://journals.sagepub.com/doi/10.1177/1748006X221131128)

5. **Cai et al. (2024)**  
   *Knowledge graph‑driven equipment fault diagnosis method for intelligent manufacturing*  
   [Int J Adv Manufacturing Technology, Vol 130](https://link.springer.com/article/10.1007/s00170-024-12998-x)

6. **Pérez Hernández (2022)**  
   *Maintenance Strategies for Networked Assets*  
   [University of Cambridge Repository (PDF)](https://www.repository.cam.ac.uk/bitstream/handle/1810/336867/Maintenance_Strategies_for_Networked_Assets.pdf)

7. **Barberá et al. (2013)**  
   *The Graphical Analysis for Maintenance Management Method (GAMM)*  
   [ResearchGate (PDF)](https://www.researchgate.net/publication/262906105_The_Graphical_Analysis_for_Maintenance_Management_Method_A_Quantitative_Graphical_Analysis_to_Support_Maintenance_Management_Decision_Making)

8. **Zheng et al. (2022)**  
   *Query-based Industrial Analytics over Knowledge Graphs with Ontology Reshaping*  

   [arXiv preprint](https://arxiv.org/abs/2209.11089)

9. **Fenza et al. (2020)**  
    *A Cognitive Approach based on the Actionable Knowledge Graph for supporting Maintenance Operations*  
    [arXiv preprint](https://arxiv.org/abs/2011.09554)

* https://github.com/jonathanwvd/awesome-industrial-datasets

* https://github.com/IBM/FailureSensorIQ/

* https://github.com/jonathanwvd/awesome-industrial-datasets/blob/master/markdown/industrial_safety_and_health_analytics_database.md

* https://github.com/jonathanwvd/awesome-industrial-datasets/blob/master/markdown/smart_manufacturing_iot-cloud_monitoring_dataset.md

* https://github.com/jonathanwvd/awesome-industrial-datasets/blob/master/markdown/productivity_prediction_of_garment_employees.md